In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
 

2022-09-08 16:44:55.546038: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-08 16:44:55.550701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/anaconda3-2021-05/lib:/cm/shared/apps/pbspro-ce/19.1.3/lib/
2022-09-08 16:44:55.550720: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## inspiration [here](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/)

In [2]:

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [8]:
def ld_data(data,n_days):

    # load dataset
    # data = read_csv('pollution.csv', header=0, index_col=0)
    values = data.values
    # integer encode direction
    encoder = LabelEncoder()
    # values[:,4] = encoder.fit_transform(values[:,4])
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    # frame as supervised learning
    # reframed = series_to_supervised(scaled, 1, 1)
    ...
    # specify the number of lag hours
    # n_days = 7
    n_features = scaled.shape[1]
    # frame as supervised learning
    reframed = series_to_supervised(scaled, n_days, 1)

    # drop columns we don't want to predict
    # reframed.drop(reframed.columns[9:], axis=1, inplace=True)

    drop=np.arange(n_features+1,(2*n_features),1)
    reframed.drop(reframed.columns[drop], axis=1, inplace=True)

    # print(reframed.head())

    # split into train and test sets
    values = reframed.values
    n_train_hours = 365 * round((len(data)/365)*.7) #5 #24
    train = values[:n_train_hours, :]
    test = values[n_train_hours:, :]
    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    # reshape input to be 3D [samples, timesteps, features]
    # train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    # test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    # print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

    # split into input and outputs
    n_obs = n_days * n_features
    train_X, train_y = train[:, :n_obs], train[:, -n_features]
    test_X, test_y = test[:, :n_obs], test[:, -n_features]
    # print(train_X.shape, len(train_X), train_y.shape)
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], n_days, n_features))
    test_X = test_X.reshape((test_X.shape[0], n_days, n_features))
    return train_X, train_y,test_X,test_y,n_features,scaler

In [5]:
def run_mdl(train_X, train_y,test_X,test_y,YY,var,site,n_days,n_features,scaler):
    # design network
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    if YY=='shuf':
        train_y=np.random.permutation(train_y)
    elif YY=='orig':
        train_y=train_y

    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=0, shuffle=True)

    # plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.title(var+'_'+YY+'_loss_'+site)
    # pyplot.show()
    plt.savefig('NNtest/'+var+'_'+YY+'_loss_'+site+'.png')
    plt.clf()
    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], n_days*n_features))

    # invert scaling for forecast
    inv_yhat = concatenate((yhat, test_X[:, -(n_features-1):]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    # calculate RMSE
    mse = (mean_squared_error(inv_y, inv_yhat))
    # print('Test MSE: %.3f' % mse)

    plt.plot(np.rint(inv_y), alpha = .5,label='pred')
    plt.plot(np.double(inv_yhat), alpha = .5,label='valid')
    plt.title(site+'_'+var+'_'+YY+ 'MSE: '+str(mse))
    plt.legend()
    plt.savefig('NNtest/'+site+'_'+var+'_'+YY+'_mse.png')
    plt.clf()
    return mse
    

In [9]:
var_list = []
mse_list = []
site_list = []
order_list = []

for i in ['all','ae','poll']:
    for SS in ['orig','shuf']:
        d6=pd.read_parquet('~/run/AXA_AE_app/AE_AXA_poll-ae.parquet')#, compression='GZIP')#.mode('overwrite')
        # site=np.random.randint(0,len(np.unique(d6['cd9_loc'])))
        # site=np.unique(d6['cd9_loc'])[site]
        # data=d6[d6['cd9_loc']==site]
        # site=[ s[0] for s in site.split() ]

        data=d6[d6['cd9_loc']=='Queen Elizabeth Hospital']
        site='QEH'
        
        data=data.groupby(['cd9_loc','date']).agg('mean')[['pm25','pm10','o3','no2','so2','co','age','sex','diag1']]
        data.age=np.round(data.age)
        data.sex=np.round(data.sex)
        # values = dataset.values
        first_column = data.pop('diag1')
        data.insert(0, 'diag1', first_column)
        print
        if i=='all':
            data=data
        elif i=='ae':
            data=data[['diag1','age','sex']]
        elif i =='poll':
            data.drop(['age','sex'],axis=1,inplace=True)
        n_days=7
        train_X, train_y,test_X,test_y,n_features,scaler=ld_data(data,n_days)
        mse=run_mdl(train_X, train_y,test_X,test_y,SS,i,site,n_days,n_features,scaler)
        
        var_list.append(i)
        mse_list.append(mse)
        site_list.append(site)
        order_list.append(SS)
        print(site+'_'+i+'_'+SS+ 'MSE: '+str(mse))
        
df = pd.DataFrame(list(zip(var_list , mse_list,site_list,order_list)), 
           columns =['var', ',mse','site','order'])

df.to_csv('NNtest/NN_mse.txt',sep='\t')

6/6 [==============================] - 0s 2ms/step
QEH_all_origMSE: 0.0018554541
6/6 [==============================] - 0s 2ms/step
QEH_all_shufMSE: 2.8838868
25/25 [==============================] - 0s 2ms/step
QEH_ae_origMSE: 0.0037106762
25/25 [==============================] - 0s 2ms/step
QEH_ae_shufMSE: 3.5277917
6/6 [==============================] - 0s 2ms/step
QEH_poll_origMSE: 0.0078683235
6/6 [==============================] - 0s 2ms/step
QEH_poll_shufMSE: 2.916048


<Figure size 432x288 with 0 Axes>

In [10]:
df

,var,",mse",site,order
0,all,0.001855,QEH,orig
1,all,2.883887,QEH,shuf
2,ae,0.003711,QEH,orig
3,ae,3.527792,QEH,shuf
4,poll,0.007868,QEH,orig
5,poll,2.916048,QEH,shuf
